<a href="https://colab.research.google.com/github/KsushaKhadzhinova/superstore/blob/main/etl_Initial_Load%2C_Secondary_Load_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/KsushaKhadzhinova/superstore

Cloning into 'superstore'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 44 (delta 13), reused 39 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (44/44), 4.92 MiB | 43.81 MiB/s, done.
Resolving deltas: 100% (13/13), done.


imports + save path

In [ ]:
import pandas as pd
import numpy as np
import os
from google.colab import files

# Path for saving files on your computer
save_folder = r"E:/intern/etl + power bi"
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

load + preprocess data



1.   Clean data (handle missing values, remove duplicates)
2.   Convert data types (e.g., dates)
3.   Encode categorical variables
4.   Normalize or scale features
5.   Remove outliers



In [ ]:
print("Upload the Superstore dataset file (CSV):")
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
print(f"File uploaded: {file_name}")


try:
    df = pd.read_csv(file_name, encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv(file_name, encoding='latin1')


print("\nFirst 5 rows:")
print(df.head())


print("\nInformation about structure and data types:")
print(df.info())


print("\nNumber of missing values per column:")
print(df.isnull().sum())


print("\nUnique values in Order Date and Ship Date (first 5):")
print(df['Order Date'].unique()[:5])
print(df['Ship Date'].unique()[:5])


df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce', infer_datetime_format=True)
df['Ship Date'] = pd.to_datetime(df['Ship Date'], errors='coerce', infer_datetime_format=True)


print("\nDates after conversion:")
print(df[['Order Date', 'Ship Date']].head())


Загрузите файл датасета Superstore (CSV):


Saving Sample - Superstore.csv to Sample - Superstore (8).csv
Файл загружен: Sample - Superstore (8).csv

Первые 5 строк:
   Row ID        Order ID  Order Date   Ship Date       Ship Mode Customer ID  \
0       1  CA-2016-152156   11/8/2016  11/11/2016    Second Class    CG-12520   
1       2  CA-2016-152156   11/8/2016  11/11/2016    Second Class    CG-12520   
2       3  CA-2016-138688   6/12/2016   6/16/2016    Second Class    DV-13045   
3       4  US-2015-108966  10/11/2015  10/18/2015  Standard Class    SO-20335   
4       5  US-2015-108966  10/11/2015  10/18/2015  Standard Class    SO-20335   

     Customer Name    Segment        Country             City  ...  \
0      Claire Gute   Consumer  United States        Henderson  ...   
1      Claire Gute   Consumer  United States        Henderson  ...   
2  Darrin Van Huff  Corporate  United States      Los Angeles  ...   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   
4   Sean O'Donnell   Consumer  United Sta

/tmp/ipython-input-1853856825.py:24: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce', infer_datetime_format=True)
/tmp/ipython-input-1853856825.py:25: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['Ship Date'] = pd.to_datetime(df['Ship Date'], errors='coerce', infer_datetime_format=True)


saving preproced data

In [ ]:
processed_file_name = os.path.join(save_folder, "processed_superstore.csv")
df.to_csv(processed_file_name, index=False)
print(f"Saved: {processed_file_name}")

Сохранено: E:/intern/etl + power bi/processed_superstore.csv


Initial Load: function for creating an initial load
(creating an initial data slice)

In [ ]:
def create_initial_load(df, cutoff_date_str):
    cutoff_date = pd.to_datetime(cutoff_date_str)
    if df['Order Date'].dtype == 'object':
        df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')
    initial_load_df = df[df['Order Date'] < cutoff_date].copy().reset_index(drop=True)
    return initial_load_df

initial_load = create_initial_load(df, '2016-01-01')
print(f"Initial Load created, rows: {len(initial_load)}")


Initial Load создан, строк: 4095


saving Initial Load data

In [ ]:
initial_load_file = os.path.join(save_folder, "initial_load_superstore.csv")
initial_load.to_csv(initial_load_file, index=False)
print(f"Сохранено: {initial_load_file}")

Сохранено: E:/intern/etl + power bi/initial_load_superstore.csv


Secondary Load: a function for creating a secondary load with new, modified, duplicates (we create a secondary slice with changes)

In [ ]:
def create_secondary_load(df, cutoff_date_str, initial_load):
    cutoff_date = pd.to_datetime(cutoff_date_str)
    secondary_df = df[df['Order Date'] >= cutoff_date].copy().reset_index(drop=True)

    max_ord_num = initial_load['Order ID'].apply(
        lambda x: int(x.split('-')[-1]) if isinstance(x, str) and '-' in x else 0).max()

    # New records
    new_records = secondary_df.head(3).copy()
    for i in range(3):
        new_records.at[i, 'Order ID'] = f"NEW-{max_ord_num + i + 1}"
        new_records.at[i, 'Customer ID'] = f"NEW-CUST-{i + 1}"
        new_records.at[i, 'Order Date'] = cutoff_date + pd.Timedelta(days=i + 10)
    secondary_df = pd.concat([secondary_df, new_records], ignore_index=True)

    # SCD Type 1 changes
    scd1_indices = secondary_df.sample(3, random_state=42).index
    for idx in scd1_indices:
        old_segment = secondary_df.at[idx, 'Segment']
        secondary_df.at[idx, 'Segment'] = 'Corporate' if old_segment != 'Corporate' else 'Consumer'

    # SCD Type 2 changes
    scd2_samples = secondary_df.sample(2, random_state=24).copy()
    for i, idx in enumerate(scd2_samples.index):
        row = secondary_df.loc[idx]
        new_row = row.copy()
        new_row['Order ID'] = f"{row['Order ID']}_ver2"
        new_row['Customer ID'] = f"{row['Customer ID']}_ver2"
        new_row['City'] = 'NewCity'
        new_row['Order Date'] = row['Order Date'] + pd.Timedelta(days=30 + i * 5)
        secondary_df = pd.concat([secondary_df, pd.DataFrame([new_row])], ignore_index=True)

    # Duplicates from Initial Load
    duplicates = initial_load.sample(2, random_state=7)
    secondary_df = pd.concat([secondary_df, duplicates], ignore_index=True)

    secondary_df.reset_index(drop=True, inplace=True)
    return secondary_df

secondary_load = create_secondary_load(df, '2016-01-01', initial_load)
print(f"Secondary Load created, rows: {len(secondary_load)}")


Secondary Load создан, строк: 5906


saving Secondary Load data

In [ ]:
secondary_load_file = os.path.join(save_folder, "secondary_load_superstore.csv")
secondary_load.to_csv(secondary_load_file, index=False)
print(f"Saved: {secondary_load_file}")

Сохранено: E:/intern/etl + power bi/secondary_load_superstore.csv
